In [3]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy


from moscot.problems.time._lineage import TemporalProblem

/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [5]:
Path="/home/mgander/moscot-framework_reproducibility/data/mouse_temporal"
ts=['E3.5', 'E4.5', 'E5.25', 'E5.5', 'E6.25', 'E6.5', 'E6.75', 'E7.0', 'E7.25', 'E7.5', 'E7.75', 'E8.0', 'E8.25', 'E8.5a', 'E8.5b', 'E9.5', 'E10.5', 'E11.5', 'E12.5', 'E13.5']
cells=[32, 80, 100, 120, 400, 660, 1720, 4500, 8200, 15000, 30000, 60000, 73000, 90000, 90000, 200000, 1100000, 2600000, 6000000, 13000000]
Cell_number_dict={}
for i in range(20):
    Cell_number_dict[ts[i]]=cells[i]

# Moscot maps to growth rates

In [6]:
# Doesn't matter which adata you use to create a tp instance, the result is always the same. I just need one
# to instantiate a tp-problem
adata=sc.read(f"{Path}/anndatas/Comb_anndatas/adata_E3.5_E4.5.h5ad")
tp=TemporalProblem(adata)

Only considering the two last: ['.5', '.h5ad'].
Only considering the two last: ['.5', '.h5ad'].


In [8]:
corm=[]
cort=[]
ns=[]
for i in range(13):
    ts0=ts[i]
    ts1=ts[i+1]
    print(f'{ts0} --> {ts1}')
    print('------------------------')
    tp=tp.load(f'{Path}/moscot_maps/{ts0}_{ts1}_no_gr2_TemporalProblem.pkl')
    adata=tp.adata
    time_tuple=list(tp.solutions.keys())[0]
    gr=tp[time_tuple].solution.a
    gr=gr/gr.mean()*Cell_number_dict[ts1]/Cell_number_dict[ts0]
    
    
    A0=adata[adata.obs['day']==time_tuple[0]].copy()
    A0.obs['gr']=gr

    grT=np.load(f'{Path}/TOME_transport_maps/{ts0}_growth_rates2.npy')
    A0.obs['grT']=grT
    
    scanpy.pp.scale(A0)
    cc_genes_mouse=list(pd.read_csv(f'{Path}/utils/mouse_proliferation.txt', header=None)[0])
    scanpy.tl.score_genes(A0, cc_genes_mouse)

    #A0.obs['gr_init']=list(tp.prior_growth_rates['prior_growth_rates'])[:len(A0)]
    A0.obs['gr_init']=A0.obs['score']
    
    corm.append(scipy.stats.pearsonr(x=A0.obs['gr'], y=A0.obs['gr_init'])[0])
    cort.append(scipy.stats.pearsonr(x=A0.obs['grT'], y=A0.obs['gr_init'])[0])
    ns.append(len(A0))

E3.5 --> E4.5
------------------------
E4.5 --> E5.25
------------------------
E5.25 --> E5.5
------------------------
E5.5 --> E6.25
------------------------
E6.25 --> E6.5
------------------------
E6.5 --> E6.75
------------------------
E6.75 --> E7.0
------------------------
E7.0 --> E7.25
------------------------
E7.25 --> E7.5
------------------------
E7.5 --> E7.75
------------------------
E7.75 --> E8.0
------------------------
E8.0 --> E8.25
------------------------
E8.25 --> E8.5a
------------------------
